# Reflection

In the context of LLM agent building, reflection refers to the process of prompting an LLM to observe its past steps (along with potential observations from tools/the environment) to assess the quality of the chosen actions. This is then used downstream for things like re-planning, search, or evaluation.

This notebook demonstrates a very simple form of reflection in LangGraph.


![](./images/Basic%20Reflection.png)

In [4]:
%pip install -U --quiet  langgraph langchain-fireworks tavily-python langchain_openai


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Generate
For our example, we will create a "5 paragraph essay" generator. First, create the generator:


In [9]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatOpenAI(
    max_tokens=4096,
)
generate = prompt | llm


In [10]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content


"The Little Prince" by Antoine de Saint-Exupéry continues to resonate with modern childhood due to its timeless themes and universal messages. This beloved novella explores profound ideas such as the importance of imagination, the value of human connections, and the need to prioritize what truly matters in life. In today's fast-paced and technology-driven world, these themes are more relevant than ever, making "The Little Prince" a poignant and essential read for children.

Firstly, the theme of imagination is crucial in the upbringing of children in the modern era. With the prevalence of screens and digital devices, there is a growing concern about the diminishing space for creativity and imagination in children's lives. "The Little Prince" encourages readers to nurture their imagination, to see beyond the surface, and to appreciate the beauty in the world around them. In a society that often values productivity over creativity, the little prince reminds children to hold onto their se

## Reflect


In [11]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [12]:
essay

'"The Little Prince" by Antoine de Saint-Exupéry continues to resonate with modern childhood due to its timeless themes and universal messages. This beloved novella explores profound ideas such as the importance of imagination, the value of human connections, and the need to prioritize what truly matters in life. In today\'s fast-paced and technology-driven world, these themes are more relevant than ever, making "The Little Prince" a poignant and essential read for children.\n\nFirstly, the theme of imagination is crucial in the upbringing of children in the modern era. With the prevalence of screens and digital devices, there is a growing concern about the diminishing space for creativity and imagination in children\'s lives. "The Little Prince" encourages readers to nurture their imagination, to see beyond the surface, and to appreciate the beauty in the world around them. In a society that often values productivity over creativity, the little prince reminds children to hold onto the

In [13]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content


Your essay on why "The Little Prince" is relevant in modern childhood is well-written and touches upon important themes from the novella. Here are some recommendations to enhance your essay:

1. **Provide Specific Examples**: While you mention the themes of imagination, human connections, and prioritizing what matters in life, try to include specific examples from the novella to support your points. For instance, you could discuss the little prince's interactions with the fox to illustrate the importance of forming meaningful relationships.

2. **Expand on the Impact of Technology**: You briefly mention the impact of technology on children's creativity and relationships, but you could delve deeper into this topic. How does the novella's message about imagination resonate in a world dominated by technology? How can children apply the little prince's lessons to balance their digital lives with real-world experiences?

3. **Discuss the Relevance of the Little Prince's Journey**: You touch

## Repeat

And... that's all there is too it! You can repeat in a loop for a fixed number of steps, or use an LLM (or other check) to decide when the finished product is good enough

In [14]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

"The Little Prince" by Antoine de Saint-Exupéry continues to resonate with modern childhood due to its timeless themes and universal messages. This beloved novella explores profound ideas such as the importance of imagination, the value of human connections, and the need to prioritize what truly matters in life. In today's fast-paced and technology-driven world, these themes are more relevant than ever, making "The Little Prince" a poignant and essential read for children.

Specific examples from the novella highlight the importance of these themes. For instance, the little prince's interactions with the fox illustrate the significance of forming meaningful relationships. Through the prince's journey, children learn about empathy, friendship, and the beauty of genuine connections.

In a world dominated by technology, the novella's message about imagination takes on added importance. Children today are often consumed by screens and digital devices, leaving little room for creativity and

## Define graph
Now that we've shown each step in isolation, we can wire it up in a graph.



In [15]:
from typing import List, Sequence

from langgraph.graph import END, MessageGraph


async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")


def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [16]:
async for event in graph.astream(
    [
        HumanMessage(
            content="Generate an essay on the topicality of The Little Prince and its message in modern life"
        )
    ],
):
    print(event)
    print("---")


{'generate': AIMessage(content="The Little Prince, written by Antoine de Saint-Exupéry, continues to captivate readers with its timeless message and relevance in modern life. This beloved novella, first published in 1943, explores profound themes such as love, friendship, loneliness, and the complexities of human nature. Despite being written over half a century ago, The Little Prince remains as poignant and thought-provoking today as it was when it was first penned.\n\nOne of the central messages of The Little Prince is the importance of seeing beyond the surface to discover the true essence of things. In a world that is often preoccupied with material possessions and superficial appearances, the story reminds us to look with the heart. The Little Prince teaches us that what is essential is invisible to the eye, emphasizing the value of love, kindness, and human connections over material wealth.\n\nMoreover, The Little Prince highlights the significance of maintaining a childlike sens

## Conclusion
Now that you've applied reflection to an LLM agent, I'll note one thing: self-reflection is inherently cyclic: it is much more effective if the reflection step has additional context or feedback (from tool observations, checks, etc.). If, like in the scenario above, the reflection step simply prompts the LLM to reflect on its output, it can still benefit the output quality (since the LLM then has multiple "shots" at getting a good output), but it's less guaranteed.

